# Abbulu - Capstone Project - Battle of Neighborhoods - Week 2

In [1]:
# import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0



geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving envir

## 1. Explore and Prepare data

In [2]:
# San Francisco Neighborhoods list
sfhoods = ['Ashbury','Balboa Park','Bayview','Bernal Heights','Castro','Chinatown','Civic Center','Cow Hollow','Dolores Heights',
'Embarcadero','Fillmore','Financial District','Fishermans Wharf','Golden Gate Heights','Haight-Ashbury','Hayes Valley','Ingleside',
'Japantown','Laurel Heights','Little Russia','Little Saigon','Mission Bay','Mission District','Nob Hill','Noe Valley',
'North Beach','Pacific Heights','Portola','Potrero Hill','Presidio','Richmond District','Russian Hill','Sunset District',
'Telegraph Hill','Tenderloin','Treasure Island','Twin Peaks','Union Square','Western Addition','Yerba Buena']

print(sfhoods)

['Ashbury', 'Balboa Park', 'Bayview', 'Bernal Heights', 'Castro', 'Chinatown', 'Civic Center', 'Cow Hollow', 'Dolores Heights', 'Embarcadero', 'Fillmore', 'Financial District', 'Fishermans Wharf', 'Golden Gate Heights', 'Haight-Ashbury', 'Hayes Valley', 'Ingleside', 'Japantown', 'Laurel Heights', 'Little Russia', 'Little Saigon', 'Mission Bay', 'Mission District', 'Nob Hill', 'Noe Valley', 'North Beach', 'Pacific Heights', 'Portola', 'Potrero Hill', 'Presidio', 'Richmond District', 'Russian Hill', 'Sunset District', 'Telegraph Hill', 'Tenderloin', 'Treasure Island', 'Twin Peaks', 'Union Square', 'Western Addition', 'Yerba Buena']


In [3]:
# Seattle Neighborhoods list
stlhoods =['Belltown','Broadmoor','Capitol Hill','Denny-Blaine','Downtown','Eastlake','First Hill','Leschi','Madison Park',
    'Madrona','Montlake','Pioneer Square','South Lake Union','Bryant','Hawthorne Hills','Lake City','Laurelhurst','Maple Leaf',
    'Matthews Beach','Northgate','Ravenna','Roosevelt','U-District','View Ridge','Wedgwood','Windermere','Ballard','Broadview',
    'Crown Hill','Fremont','Green Lake','Greenwood','Haller Lake','Loyal Heights','Magnolia','Phinney Ridge','Queen Anne',
    'Wallingford','Westlake','Whittier Heights','Beacon Hill','Columbia City','Georgetown','Mount Baker','Rainier Beach',
    'Seward Park','South Park']

print(stlhoods)

['Belltown', 'Broadmoor', 'Capitol Hill', 'Denny-Blaine', 'Downtown', 'Eastlake', 'First Hill', 'Leschi', 'Madison Park', 'Madrona', 'Montlake', 'Pioneer Square', 'South Lake Union', 'Bryant', 'Hawthorne Hills', 'Lake City', 'Laurelhurst', 'Maple Leaf', 'Matthews Beach', 'Northgate', 'Ravenna', 'Roosevelt', 'U-District', 'View Ridge', 'Wedgwood', 'Windermere', 'Ballard', 'Broadview', 'Crown Hill', 'Fremont', 'Green Lake', 'Greenwood', 'Haller Lake', 'Loyal Heights', 'Magnolia', 'Phinney Ridge', 'Queen Anne', 'Wallingford', 'Westlake', 'Whittier Heights', 'Beacon Hill', 'Columbia City', 'Georgetown', 'Mount Baker', 'Rainier Beach', 'Seward Park', 'South Park']


## 1. Get Neighborhood data for San Francisco

In [5]:
# Initialize dataframes and variables
sanfran = 'San Francisco, CA'
seattle = 'Seattle, WA'

# create data frame for San Francisco data
column_names = ['City','Neighborhood','Latitude','Longitude'] 

# instantiate the dataframes
dfsfhoods = pd.DataFrame(columns=column_names,index=['Neighborhood'])
dfstlhoods = pd.DataFrame(columns=column_names,index=['Neighborhood'])

In [6]:
# fill the dataframe with San Francisco neighborhoods and latitude and longitude detils
geolocator = Nominatim(user_agent="sf_explorer")

for hood in sfhoods:
    address = hood + ', ' + sanfran
    
    
    try:
        location = geolocator.geocode(address)
    except: 
        print(address, ' address can not be geo located')
        
    try:
        latitude = location.latitude
        latfound = True
    except:
        latfound = False
        
    try:
        longitude = location.longitude
        longfound = True
    except:
        longfound = False
    
    if latfound & longfound:
        #print('     The geograpical coordinate for {}, are, {}, {}.'.format(address, latitude, longitude))
        dfsfhoods = dfsfhoods.append({'City':'San Francisco', 
                                          'Neighborhood': hood,
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)

In [7]:
# cleanup neighborhood data
dfsfhoods.drop(dfsfhoods.loc[dfsfhoods['Neighborhood'] == 'NaN'].index, inplace=True)
dfsfhoods.drop([0], inplace=True)

dfsfhoods.drop_duplicates(inplace=True)

dfsfhoods = dfsfhoods.reset_index(drop=True)

dfsfhoods.sort_values(['Neighborhood'], ascending=[True])

dfsfhoods = dfsfhoods.reset_index(drop=True)

dfsfhoods

,City,Neighborhood,Latitude,Longitude
0,San Francisco,Ashbury,37.767133,-122.446520
1,San Francisco,Balboa Park,37.724949,-122.444805
2,San Francisco,Bayview,37.728889,-122.392500
3,San Francisco,Bernal Heights,37.741001,-122.414214
4,San Francisco,Castro,37.760856,-122.434957
5,San Francisco,Chinatown,37.794301,-122.406376
6,San Francisco,Civic Center,37.779594,-122.416794
7,San Francisco,Cow Hollow,37.797262,-122.436248
8,San Francisco,Dolores Heights,37.745435,-122.424480
9,San Francisco,Embarcadero,37.792864,-122.396912


In [8]:
#get shape of the data frame
dfsfhoods.shape

(37, 4)

In [9]:
# get latitude and longitude for San Francisco
geolocator = Nominatim(user_agent="sf_explorer")
address = 'San Francisco, CA'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7792808, -122.4192363.


In [10]:
# create map of San Francisco using latitude and longitude values
map_sanfran = folium.Map(location=[latitude, longitude], zoom_start=10)

In [22]:
# add markers to map
for lat, lng, neighborhood in zip(dfsfhoods['Latitude'], dfsfhoods['Longitude'], dfsfhoods['Neighborhood']):
    
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sanfran)  
    
map_sanfran

In [1]:
# define Foursquare Credentials and Version
# REMOVED

In [24]:
# print data frame details
print('The dataframe has {} neighborhoods.'.format(dfsfhoods.shape[0]))

The dataframe has 37 neighborhoods.


In [25]:
# explore the first neighborhood in the dataframe.
dfsfhoods.loc[0:0, 'Neighborhood']

0    Ashbury
Name: Neighborhood, dtype: object

In [26]:
# get the neighborhood's latitude and longitude values.
neighborhood_latitude = dfsfhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfsfhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = dfsfhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Ashbury are 37.7671329, -122.4465199.


In [27]:
# set url for the FourSquare API

radius = 500
LIMIT=100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude,
    neighborhood_longitude,
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=Z1YIEDJDV2ZAEGSSF33UOQ4JEAEXKRDDSL22ZXXSHHZ4ESJI&client_secret=4T3IT1KKLJVBB54JPXZAU3P4EFQ3MGICY0HI1LMK5NNDVAGO&v=20190615&ll=37.7671329,-122.4465199&radius=500&limit=100'

In [28]:
# send the GET request to FourSquare and examine the results
results = requests.get(url).json()

In [19]:
# show results
results

In [29]:
# define function that extracts the category of the venue from FourSquare lab
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

#And how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in San Francisco

In [31]:
# create a function to repeat the clean the json structure process to all the neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
# run the function getNearbyVenues(names, latitudes, longitudes, radius=500)
# on each neighborhood and create a new dataframe called sf_venues

sf_venues = getNearbyVenues(names=dfsfhoods['Neighborhood'],
                                   latitudes=dfsfhoods['Latitude'],
                                   longitudes=dfsfhoods['Longitude']
                                  )

Ashbury
Balboa Park
Bayview
Bernal Heights
Castro
Chinatown
Civic Center
Cow Hollow
Dolores Heights
Embarcadero
Fillmore
Financial District
Fishermans Wharf
Haight-Ashbury
Hayes Valley
Ingleside
Japantown
Little Saigon
Mission Bay
Mission District
Nob Hill
Noe Valley
North Beach
Pacific Heights
Portola
Potrero Hill
Presidio
Richmond District
Russian Hill
Sunset District
Telegraph Hill
Tenderloin
Treasure Island
Twin Peaks
Union Square
Western Addition
Yerba Buena


In [33]:
# check the size of the resulting dataframe
print(sf_venues.shape)

(2353, 7)


In [34]:
# check how many venues were returned for each neighborhood
sf_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ashbury,100,100,100,100,100,100
Balboa Park,14,14,14,14,14,14
Bayview,13,13,13,13,13,13
Bernal Heights,43,43,43,43,43,43
Castro,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,97,97,97,97,97,97
Cow Hollow,100,100,100,100,100,100
Dolores Heights,91,91,91,91,91,91


In [35]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(sf_venues['Venue Category'].unique())))

There are 281 uniques categories.


## 3. Analyze each Neighborhood in San Francisco

In [36]:
# use one hot encoding to be used to create clusters
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sf_onehot['Neighborhood'] = sf_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Yoga Studio,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Farmers Market,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Monument / Landmark,Motel,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Reservoir,Restaurant,Rock Club,Roof Deck,Rugby Pitch,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Sicilian Restaurant,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Trattoria/Osteria,Turkish Restaurant,Tuscan Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Whisky Bar

In [37]:
# examine the new dataframe size
sf_onehot.shape

(2353, 281)

In [38]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped

,Neighborhood,Yoga Studio,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Farmers Market,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Monument / Landmark,Motel,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Reservoir,Restaurant,Rock Club,Roof Deck,Rugby Pitch,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Sicilian Restaurant,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Trattoria/Osteria,Turkish Restaurant,Tuscan Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Whisky Bar

In [39]:
#confirm the new dataframe size
sf_grouped.shape

(36, 281)

In [40]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in sf_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sf_grouped[sf_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ashbury----
                    venue  freq
0             Coffee Shop  0.07
1                Boutique  0.06
2          Clothing Store  0.05
3  Thrift / Vintage Store  0.05
4               Gift Shop  0.04


----Balboa Park----
                venue  freq
0  Light Rail Station  0.14
1       Metro Station  0.07
2         Flower Shop  0.07
3        Tennis Court  0.07
4        Dessert Shop  0.07


----Bayview----
                             venue  freq
0                           Bakery  0.15
1               Mexican Restaurant  0.15
2  Southern / Soul Food Restaurant  0.15
3                     Dance Studio  0.08
4                             Café  0.08


----Bernal Heights----
          venue  freq
0   Coffee Shop  0.07
1        Bakery  0.07
2    Playground  0.05
3    Food Truck  0.05
4  Gourmet Shop  0.05


----Castro----
                     venue  freq
0                  Gay Bar  0.10
1              Coffee Shop  0.05
2  New American Restaurant  0.03
3          Thai Restaurant  0.03

In [41]:
# write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
# create the new dataframe and display the top 10 venues for each neighborhood using the 
# function return_most_common_venues

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ashbury,Coffee Shop,Boutique,Clothing Store,Thrift / Vintage Store,Gift Shop,Breakfast Spot,Shoe Store,Bar,Bookstore,Pizza Place
1,Balboa Park,Light Rail Station,Bus Line,Metro Station,Sandwich Place,BBQ Joint,Park,Asian Restaurant,Flower Shop,Dessert Shop,Tennis Court
2,Bayview,Southern / Soul Food Restaurant,Mexican Restaurant,Bakery,Pool,Pharmacy,Gym,Café,Food Truck,Dance Studio,Piercing Parlor
3,Bernal Heights,Coffee Shop,Bakery,Italian Restaurant,Park,Gourmet Shop,Food Truck,Playground,Peruvian Restaurant,New American Restaurant,Mexican Restaurant
4,Castro,Gay Bar,Coffee Shop,New American Restaurant,Pet Store,Thai Restaurant,Yoga Studio,Art Gallery,Gym,Mediterranean Restaurant,Men's Store


## 4. Cluster Neighborhoods

In [43]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5
sf_grouped_clustering = sf_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Ashbury,Coffee Shop,Boutique,Clothing Store,Thrift / Vintage Store,Gift Shop,Breakfast Spot,Shoe Store,Bar,Bookstore,Pizza Place
1,0,Balboa Park,Light Rail Station,Bus Line,Metro Station,Sandwich Place,BBQ Joint,Park,Asian Restaurant,Flower Shop,Dessert Shop,Tennis Court
2,1,Bayview,Southern / Soul Food Restaurant,Mexican Restaurant,Bakery,Pool,Pharmacy,Gym,Café,Food Truck,Dance Studio,Piercing Parlor
3,1,Bernal Heights,Coffee Shop,Bakery,Italian Restaurant,Park,Gourmet Shop,Food Truck,Playground,Peruvian Restaurant,New American Restaurant,Mexican Restaurant
4,0,Castro,Gay Bar,Coffee Shop,New American Restaurant,Pet Store,Thai Restaurant,Yoga Studio,Art Gallery,Gym,Mediterranean Restaurant,Men's Store
5,0,Chinatown,Chinese Restaurant,Coffee Shop,Bakery,Men's Store,New American Restaurant,Hotel,Dim Sum Restaurant,Cocktail Bar,Szechuan Restaurant,Tea Room
6,0,Civic Center,Coffee Shop,Vietnamese Restaurant,Cocktail Bar,Beer Bar,Hotel,Café,Sandwich Place,Performing Arts Venue,Vegetarian / Vegan Restaurant,Theater
7,0,Cow Hollow,Cosmetics Shop,French Restaurant,Wine Bar,Gym / Fitness Center,Italian Restaurant,Salad Place,Sushi Restaurant,Gym,Thai Restaurant,Spa
8,0,Dolores Heights,Chinese Restaurant,Breakfast Spot,Pizza Place,Coffee Shop,Italian Restaurant,Mexican Restaurant,Cocktail Bar,Café,Gym / Fitness Center,Sushi Restaurant
9,0,Embarcadero,Coffee Shop,Food Truck,Sandwich Place,Japanese Restaurant,Café,Seafood Restaurant,Gym,Vegetarian / Vegan Restaurant,Deli / Bodega,Salad Place


In [45]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sf_merged = dfsfhoods

# merge sf_grouped with cluster data
sf_merged = sf_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
sf_merged.dropna(how='any',inplace=True)
sf_merged = sf_merged.reset_index(drop=True)
sf_merged['Cluster Labels'] = sf_merged['Cluster Labels'].astype(int)
sf_merged

,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,San Francisco,Ashbury,37.767133,-122.446520,0,Coffee Shop,Boutique,Clothing Store,Thrift / Vintage Store,Gift Shop,Breakfast Spot,Shoe Store,Bar,Bookstore,Pizza Place
1,San Francisco,Balboa Park,37.724949,-122.444805,0,Light Rail Station,Bus Line,Metro Station,Sandwich Place,BBQ Joint,Park,Asian Restaurant,Flower Shop,Dessert Shop,Tennis Court
2,San Francisco,Bayview,37.728889,-122.392500,1,Southern / Soul Food Restaurant,Mexican Restaurant,Bakery,Pool,Pharmacy,Gym,Café,Food Truck,Dance Studio,Piercing Parlor
3,San Francisco,Bernal Heights,37.741001,-122.414214,1,Coffee Shop,Bakery,Italian Restaurant,Park,Gourmet Shop,Food Truck,Playground,Peruvian Restaurant,New American Restaurant,Mexican Restaurant
4,San Francisco,Castro,37.760856,-122.434957,0,Gay Bar,Coffee Shop,New American Restaurant,Pet Store,Thai Restaurant,Yoga Studio,Art Gallery,Gym,Mediterranean Restaurant,Men's Store
5,San Francisco,Chinatown,37.794301,-122.406376,0,Chinese Restaurant,Coffee Shop,Bakery,Men's Store,New American Restaurant,Hotel,Dim Sum Restaurant,Cocktail Bar,Szechuan Restaurant,Tea Room
6,San Francisco,Civic Center,37.779594,-122.416794,0,Coffee Shop,Vietnamese Restaurant,Cocktail Bar,Beer Bar,Hotel,Café,Sandwich Place,Performing Arts Venue,Vegetarian / Vegan Restaurant,Theater
7,San Francisco,Cow Hollow,37.797262,-122.436248,0,Cosmetics Shop,French Restaurant,Wine Bar,Gym / Fitness Center,Italian Restaurant,Salad Place,Sushi Restaurant,Gym,Thai Restaurant,Spa
8,San Francisco,Dolores Heights,37.745435,-122.424480,0,Chinese Restaurant,Breakfast Spot,Pizza Place,Coffee Shop,Italian Restaurant,Mexican Restaurant,Cocktail Bar,Café,Gym / Fitness Center,Sushi Restaurant
9,San Francisco,Embarcadero,37.792864,-122.396912,0,Coffee Shop,Food Truck,Sandwich Place,Japanese Restaurant,Café,Seafood Restaurant,Gym,Vegetarian / Vegan Restaurant,Deli / Bodega,Salad Place


In [46]:
# visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [47]:
# examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
# Based on the defining categories, assign a name to each cluster. 

# cluster1 - It's what makes San Francisco
sf_merged.loc[sf_merged['Cluster Labels'] == 0, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ashbury,Coffee Shop,Boutique,Clothing Store,Thrift / Vintage Store,Gift Shop,Breakfast Spot,Shoe Store,Bar,Bookstore,Pizza Place
1,Balboa Park,Light Rail Station,Bus Line,Metro Station,Sandwich Place,BBQ Joint,Park,Asian Restaurant,Flower Shop,Dessert Shop,Tennis Court
4,Castro,Gay Bar,Coffee Shop,New American Restaurant,Pet Store,Thai Restaurant,Yoga Studio,Art Gallery,Gym,Mediterranean Restaurant,Men's Store
5,Chinatown,Chinese Restaurant,Coffee Shop,Bakery,Men's Store,New American Restaurant,Hotel,Dim Sum Restaurant,Cocktail Bar,Szechuan Restaurant,Tea Room
6,Civic Center,Coffee Shop,Vietnamese Restaurant,Cocktail Bar,Beer Bar,Hotel,Café,Sandwich Place,Performing Arts Venue,Vegetarian / Vegan Restaurant,Theater
7,Cow Hollow,Cosmetics Shop,French Restaurant,Wine Bar,Gym / Fitness Center,Italian Restaurant,Salad Place,Sushi Restaurant,Gym,Thai Restaurant,Spa
8,Dolores Heights,Chinese Restaurant,Breakfast Spot,Pizza Place,Coffee Shop,Italian Restaurant,Mexican Restaurant,Cocktail Bar,Café,Gym / Fitness Center,Sushi Restaurant
9,Embarcadero,Coffee Shop,Food Truck,Sandwich Place,Japanese Restaurant,Café,Seafood Restaurant,Gym,Vegetarian / Vegan Restaurant,Deli / Bodega,Salad Place
10,Fillmore,Cosmetics Shop,Italian Restaurant,Boutique,Ice Cream Shop,Park,Sandwich Place,Asian Restaurant,Bakery,Café,Coffee Shop
11,Financial District,Coffee Shop,Café,Sandwich Place,Food Truck,Gym / Fitness Center,Gym,Japanese Restaurant,Juice Bar,Deli / Bodega,Men's Store


In [48]:
# cluster2 - Hills of San Francisco
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bayview,Southern / Soul Food Restaurant,Mexican Restaurant,Bakery,Pool,Pharmacy,Gym,Café,Food Truck,Dance Studio,Piercing Parlor
3,Bernal Heights,Coffee Shop,Bakery,Italian Restaurant,Park,Gourmet Shop,Food Truck,Playground,Peruvian Restaurant,New American Restaurant,Mexican Restaurant
20,Nob Hill,Italian Restaurant,Hotel,Bar,Café,Coffee Shop,Wine Bar,French Restaurant,Spa,American Restaurant,Cocktail Bar
22,North Beach,Italian Restaurant,Pizza Place,Bakery,Cocktail Bar,Trail,Scenic Lookout,Sandwich Place,Park,Dive Bar,Deli / Bodega
28,Russian Hill,Italian Restaurant,Wine Bar,Sushi Restaurant,Park,Café,Chinese Restaurant,Theater,French Restaurant,Pizza Place,Gym
30,Telegraph Hill,Italian Restaurant,Pizza Place,Sandwich Place,Bakery,Scenic Lookout,Park,Deli / Bodega,Trail,Café,Clothing Store


In [49]:
# cluster3 - TwinPeaks tourist place
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Twin Peaks,Trail,Scenic Lookout,Tailor Shop,Garden,Bus Stop,Reservoir,Farmers Market,Electronics Store,Elementary School,Ethiopian Restaurant


In [50]:
# cluster4 - Asian Restaurants
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Portola,Vietnamese Restaurant,Sandwich Place,Bakery,Chinese Restaurant,Playground,Dim Sum Restaurant,Brewery,Grocery Store,Doctor's Office,BBQ Joint
27,Richmond District,Bakery,Chinese Restaurant,Asian Restaurant,Grocery Store,Sushi Restaurant,Korean Restaurant,Mexican Restaurant,Bubble Tea Shop,Nail Salon,Marijuana Dispensary
29,Sunset District,Chinese Restaurant,Dim Sum Restaurant,Bubble Tea Shop,Ice Cream Shop,Bus Line,Bar,Liquor Store,Taiwanese Restaurant,Grocery Store,Middle School


In [51]:
# cluster5 - Just Treasure Island
sf_merged.loc[sf_merged['Cluster Labels'] == 4, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Treasure Island,Baseball Field,Athletics & Sports,Flea Market,Rugby Pitch,Island,Gym,American Restaurant,Grocery Store,Ethiopian Restaurant,Event Space


# Explore Seattle Neighborhoods

## 1. Get Seattle Neighborhood data

In [52]:
# get longitude and latitude date for Seattle
geolocator = Nominatim(user_agent="stl_explorer")

for hood in stlhoods:
    address = hood + ', ' + seattle
    
    try:
        location = geolocator.geocode(address)
    except:
        print('geo location could not be found')
        
    try:
        latitude = location.latitude
        latfound = True
    except:
        latfound = False
        
    try:
        longitude = location.longitude
        longfound = True
    except:
        longfound = False
    
    if latfound & longfound:
        #print('     The geograpical coordinate for {}, are, {}, {}.'.format(address, latitude, longitude))
        dfstlhoods = dfstlhoods.append({'City':'Seattle', 
                                          'Neighborhood': hood,
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)

In [53]:
# cleanup data list seattle neighborhoods with geospatial data
dfstlhoods.drop([0], inplace=True)
dfstlhoods = dfstlhoods.reset_index(drop=True)
dfstlhoods

,City,Neighborhood,Latitude,Longitude
0,Seattle,Belltown,47.613231,-122.345361
1,Seattle,Broadmoor,47.635170,-122.289711
2,Seattle,Capitol Hill,47.623831,-122.318369
3,Seattle,Denny-Blaine,47.622327,-122.285985
4,Seattle,Downtown,47.604872,-122.333458
5,Seattle,Eastlake,47.643145,-122.326172
6,Seattle,First Hill,47.609305,-122.324984
7,Seattle,Leschi,47.600535,-122.291866
8,Seattle,Madison Park,47.635930,-122.280196
9,Seattle,Madrona,47.612792,-122.291226


In [54]:
# get shape of the data frame
dfstlhoods.shape

(47, 4)

In [55]:
# set geolocator for Seattle
geolocator = Nominatim(user_agent="stl_explorer")
address = 'Seattle, WA'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Seattle are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Seattle are 47.6038321, -122.3300624.


In [56]:
# create map of Seattle using latitude and longitude values
map_seattle = folium.Map(location=[latitude, longitude], zoom_start=10)

In [65]:
# add markers to map
for lat, lng, neighborhood in zip(dfstlhoods['Latitude'], dfstlhoods['Longitude'], dfstlhoods['Neighborhood']):
    
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seattle)  
    
map_seattle

In [2]:
# define Foursquare Credentials and Version
# REMOVED

In [59]:
# print data frame details
print('The dataframe has {} neighborhoods.'.format(dfstlhoods.shape[0]))

The dataframe has 47 neighborhoods.


In [60]:
# explore the first neighborhood in the dataframe.
dfstlhoods.loc[0:0, 'Neighborhood']

0    Belltown
Name: Neighborhood, dtype: object

In [66]:
# get the neighborhood's latitude and longitude values.
neighborhood_latitude = dfstlhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfstlhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = dfstlhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Belltown are 47.6132311, -122.3453607.


In [67]:
# set url for the FourSquare API call
radius = 500
LIMIT=100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude,
    neighborhood_longitude,
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=Z1YIEDJDV2ZAEGSSF33UOQ4JEAEXKRDDSL22ZXXSHHZ4ESJI&client_secret=4T3IT1KKLJVBB54JPXZAU3P4EFQ3MGICY0HI1LMK5NNDVAGO&v=20190615&ll=47.6132311,-122.3453607&radius=500&limit=100'

In [68]:
# send the GET request and examine the results
results = requests.get(url).json()

In [70]:
# show results
results

In [71]:
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

#And how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Seattle

In [72]:
# run the function getNearbyVenues(names, latitudes, longitudes, radius=500)
# on each neighborhood and create a new dataframe called stl_venues

stl_venues = getNearbyVenues(names=dfstlhoods['Neighborhood'],
                                 latitudes=dfstlhoods['Latitude'],
                                   longitudes=dfstlhoods['Longitude']
                                  )

Belltown
Broadmoor
Capitol Hill
Denny-Blaine
Downtown
Eastlake
First Hill
Leschi
Madison Park
Madrona
Montlake
Pioneer Square
South Lake Union
Bryant
Hawthorne Hills
Lake City
Laurelhurst
Maple Leaf
Matthews Beach
Northgate
Ravenna
Roosevelt
U-District
View Ridge
Wedgwood
Windermere
Ballard
Broadview
Crown Hill
Fremont
Green Lake
Greenwood
Haller Lake
Loyal Heights
Magnolia
Phinney Ridge
Queen Anne
Wallingford
Westlake
Whittier Heights
Beacon Hill
Columbia City
Georgetown
Mount Baker
Rainier Beach
Seward Park
South Park


In [73]:
# check the size of the resulting dataframe
print(stl_venues.shape)

(1323, 7)


In [74]:
# check how many venues were returned for each neighborhood
stl_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ballard,9,9,9,9,9,9
Beacon Hill,18,18,18,18,18,18
Belltown,100,100,100,100,100,100
Broadmoor,4,4,4,4,4,4
Broadview,7,7,7,7,7,7
Bryant,6,6,6,6,6,6
Capitol Hill,62,62,62,62,62,62
Columbia City,36,36,36,36,36,36
Crown Hill,13,13,13,13,13,13


In [75]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(stl_venues['Venue Category'].unique())))

There are 240 uniques categories.


## 3. Analyze each Neighborhood in Seattle

In [76]:
# use one hot encoding to create clusters of the neighborhoods
stl_onehot = pd.get_dummies(stl_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
stl_onehot['Neighborhood'] = stl_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [stl_onehot.columns[-1]] + list(stl_onehot.columns[:-1])
stl_onehot = stl_onehot[fixed_columns]

stl_onehot.head()

,Zoo Exhibit,ATM,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Science Building,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Trade School,Trail,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Belltown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [77]:
# examine the new dataframe size
stl_onehot.shape

(1323, 240)

In [78]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.
stl_grouped = stl_onehot.groupby('Neighborhood').mean().reset_index()
stl_grouped

,Neighborhood,Zoo Exhibit,ATM,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Science Building,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Trade School,Trail,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Ballard,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.

In [79]:
# show the new dataframe size
stl_grouped.shape

(47, 240)

In [80]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in stl_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = stl_grouped[stl_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ballard----
               venue  freq
0        Video Store  0.11
1  French Restaurant  0.11
2     Baseball Field  0.11
3                Bar  0.11
4  Food & Drink Shop  0.11


----Beacon Hill----
              venue  freq
0              Café  0.11
1            Bakery  0.06
2       Gas Station  0.06
3           Brewery  0.06
4  Greek Restaurant  0.06


----Belltown----
              venue  freq
0               Bar  0.07
1            Bakery  0.06
2       Coffee Shop  0.05
3  Sushi Restaurant  0.04
4             Hotel  0.04


----Broadmoor----
              venue  freq
0  Botanical Garden  0.25
1        Playground  0.25
2       Golf Course  0.25
3      Soccer Field  0.25
4       Zoo Exhibit  0.00


----Broadview----
                    venue  freq
0  Furniture / Home Store  0.14
1            Antique Shop  0.14
2        Sushi Restaurant  0.14
3             Pizza Place  0.14
4         Thai Restaurant  0.14


----Bryant----
                      venue  freq
0                      Café  0

In [81]:
# create the new dataframe and display the top 10 venues for each neighborhood using 
# the function return_most_common_venues

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = stl_grouped['Neighborhood']

for ind in np.arange(stl_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(stl_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ballard,Bar,Video Store,Food & Drink Shop,Gift Shop,Baseball Field,Coffee Shop,Park,Jewelry Store,French Restaurant,Drugstore
1,Beacon Hill,Café,Bakery,Mexican Restaurant,Brewery,Mediterranean Restaurant,Gas Station,Scenic Lookout,Taco Place,Sandwich Place,Coffee Shop
2,Belltown,Bar,Bakery,Coffee Shop,Hotel,Sushi Restaurant,Cocktail Bar,Italian Restaurant,Lounge,New American Restaurant,Pizza Place
3,Broadmoor,Playground,Botanical Garden,Soccer Field,Golf Course,Zoo,Event Space,Food & Drink Shop,Food,Flower Shop,Fish Market
4,Broadview,Furniture / Home Store,Antique Shop,Thai Restaurant,Sushi Restaurant,Video Store,Convenience Store,Pizza Place,Flower Shop,Fish Market,Food


## 4. Cluster Neighborhoods

In [82]:
# Run k-means to cluster the Seattle neighborhoods into 5 clusters.

# set number of clusters
kclusters = 5
stl_grouped_clustering = stl_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(stl_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 3], dtype=int32)

In [83]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Ballard,Bar,Video Store,Food & Drink Shop,Gift Shop,Baseball Field,Coffee Shop,Park,Jewelry Store,French Restaurant,Drugstore
1,2,Beacon Hill,Café,Bakery,Mexican Restaurant,Brewery,Mediterranean Restaurant,Gas Station,Scenic Lookout,Taco Place,Sandwich Place,Coffee Shop
2,2,Belltown,Bar,Bakery,Coffee Shop,Hotel,Sushi Restaurant,Cocktail Bar,Italian Restaurant,Lounge,New American Restaurant,Pizza Place
3,2,Broadmoor,Playground,Botanical Garden,Soccer Field,Golf Course,Zoo,Event Space,Food & Drink Shop,Food,Flower Shop,Fish Market
4,2,Broadview,Furniture / Home Store,Antique Shop,Thai Restaurant,Sushi Restaurant,Video Store,Convenience Store,Pizza Place,Flower Shop,Fish Market,Food
5,2,Bryant,Café,Pizza Place,Mediterranean Restaurant,Donut Shop,Grocery Store,Zoo,Falafel Restaurant,Food Truck,Food & Drink Shop,Food
6,2,Capitol Hill,Coffee Shop,Italian Restaurant,Cocktail Bar,Thai Restaurant,Taco Place,American Restaurant,Café,Yoga Studio,Mexican Restaurant,Japanese Restaurant
7,2,Columbia City,Coffee Shop,Park,African Restaurant,Pizza Place,Ice Cream Shop,Vietnamese Restaurant,Bar,Caribbean Restaurant,Bookstore,Sports Bar
8,2,Crown Hill,Coffee Shop,Sports Bar,Grocery Store,Pet Store,Sandwich Place,Bus Station,Burger Joint,Mexican Restaurant,Taco Place,Pizza Place
9,3,Denny-Blaine,Park,Monument / Landmark,Surf Spot,Beach,Fair,French Restaurant,Food Truck,Food & Drink Shop,Food,Flower Shop


In [84]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
stl_merged = dfstlhoods

# merge dfstlhoods with cluster data
stl_merged = stl_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
stl_merged['Cluster Labels'] = stl_merged['Cluster Labels'].astype(int)
stl_merged

,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Seattle,Belltown,47.613231,-122.345361,2,Bar,Bakery,Coffee Shop,Hotel,Sushi Restaurant,Cocktail Bar,Italian Restaurant,Lounge,New American Restaurant,Pizza Place
1,Seattle,Broadmoor,47.635170,-122.289711,2,Playground,Botanical Garden,Soccer Field,Golf Course,Zoo,Event Space,Food & Drink Shop,Food,Flower Shop,Fish Market
2,Seattle,Capitol Hill,47.623831,-122.318369,2,Coffee Shop,Italian Restaurant,Cocktail Bar,Thai Restaurant,Taco Place,American Restaurant,Café,Yoga Studio,Mexican Restaurant,Japanese Restaurant
3,Seattle,Denny-Blaine,47.622327,-122.285985,3,Park,Monument / Landmark,Surf Spot,Beach,Fair,French Restaurant,Food Truck,Food & Drink Shop,Food,Flower Shop
4,Seattle,Downtown,47.604872,-122.333458,2,Hotel,Coffee Shop,New American Restaurant,Cocktail Bar,Seafood Restaurant,Café,American Restaurant,Thai Restaurant,Donut Shop,Outdoor Supply Store
5,Seattle,Eastlake,47.643145,-122.326172,2,Sandwich Place,Mexican Restaurant,Coffee Shop,Playground,Bar,Deli / Bodega,Park,Burger Joint,Steakhouse,Furniture / Home Store
6,Seattle,First Hill,47.609305,-122.324984,2,Sandwich Place,Coffee Shop,Hotel,Pharmacy,Pizza Place,Asian Restaurant,Bakery,Creperie,Food & Drink Shop,Supplement Shop
7,Seattle,Leschi,47.600535,-122.291866,1,Park,Grocery Store,Playground,Pizza Place,Bus Stop,Pet Store,Farmers Market,Event Space,Fair,Falafel Restaurant
8,Seattle,Madison Park,47.635930,-122.280196,2,Bar,American Restaurant,Park,Bank,Mexican Restaurant,Pharmacy,Salon / Barbershop,Kitchen Supply Store,Scenic Lookout,Thai Restaurant
9,Seattle,Madrona,47.612792,-122.291226,2,Farmers Market,Bus Stop,Gift Shop,Asian Restaurant,Food Truck,Cupcake Shop,Shipping Store,Coffee Shop,Playground,Ethiopian Restaurant


In [85]:
# visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stl_merged['Latitude'], stl_merged['Longitude'], stl_merged['Neighborhood'], stl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [86]:
# examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
# Based on the defining categories, assign a name to each cluster. 

# cluster1 - parks and recreation
stl_merged.loc[stl_merged['Cluster Labels'] == 0, stl_merged.columns[[1] + list(range(5, stl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Haller Lake,Park,Dog Run,Lake,Coffee Shop,Baseball Field,Food Truck,Flower Shop,Food,Fish Market,Event Space
46,South Park,Dog Run,Brewery,Park,Fair,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,Flower Shop


In [87]:
# cluster2 - Restaurants and Entertainment
stl_merged.loc[stl_merged['Cluster Labels'] == 1, stl_merged.columns[[1] + list(range(5, stl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Leschi,Park,Grocery Store,Playground,Pizza Place,Bus Stop,Pet Store,Farmers Market,Event Space,Fair,Falafel Restaurant
16,Laurelhurst,Pharmacy,Music Venue,Bus Stop,Café,Gift Shop,Park,Zoo,Falafel Restaurant,Food & Drink Shop,Food
18,Matthews Beach,Park,Playground,Trail,Zoo,Fast Food Restaurant,Fair,Falafel Restaurant,Farm,Farmers Market,Fish & Chips Shop
25,Windermere,Pizza Place,Greek Restaurant,Park,Zoo,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop
34,Magnolia,Bus Stop,Park,Gym / Fitness Center,Music Venue,Garden,Dog Run,Korean Restaurant,Fast Food Restaurant,Falafel Restaurant,Farm
43,Mount Baker,Park,Thai Restaurant,Pizza Place,Harbor / Marina,Pet Store,Zoo,Farm,Fair,Falafel Restaurant,Farmers Market


In [88]:
# cluster3 - Coffee shops, and Restaurants
stl_merged.loc[stl_merged['Cluster Labels'] == 2, stl_merged.columns[[1] + list(range(5, stl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belltown,Bar,Bakery,Coffee Shop,Hotel,Sushi Restaurant,Cocktail Bar,Italian Restaurant,Lounge,New American Restaurant,Pizza Place
1,Broadmoor,Playground,Botanical Garden,Soccer Field,Golf Course,Zoo,Event Space,Food & Drink Shop,Food,Flower Shop,Fish Market
2,Capitol Hill,Coffee Shop,Italian Restaurant,Cocktail Bar,Thai Restaurant,Taco Place,American Restaurant,Café,Yoga Studio,Mexican Restaurant,Japanese Restaurant
4,Downtown,Hotel,Coffee Shop,New American Restaurant,Cocktail Bar,Seafood Restaurant,Café,American Restaurant,Thai Restaurant,Donut Shop,Outdoor Supply Store
5,Eastlake,Sandwich Place,Mexican Restaurant,Coffee Shop,Playground,Bar,Deli / Bodega,Park,Burger Joint,Steakhouse,Furniture / Home Store
6,First Hill,Sandwich Place,Coffee Shop,Hotel,Pharmacy,Pizza Place,Asian Restaurant,Bakery,Creperie,Food & Drink Shop,Supplement Shop
8,Madison Park,Bar,American Restaurant,Park,Bank,Mexican Restaurant,Pharmacy,Salon / Barbershop,Kitchen Supply Store,Scenic Lookout,Thai Restaurant
9,Madrona,Farmers Market,Bus Stop,Gift Shop,Asian Restaurant,Food Truck,Cupcake Shop,Shipping Store,Coffee Shop,Playground,Ethiopian Restaurant
10,Montlake,Bus Stop,Salon / Barbershop,Grocery Store,Coffee Shop,American Restaurant,Italian Restaurant,Library,Bike Shop,History Museum,Park
11,Pioneer Square,Coffee Shop,Hotel,Cocktail Bar,Café,Seafood Restaurant,Donut Shop,Italian Restaurant,Salad Place,Sandwich Place,Scenic Lookout


In [89]:
# cluster4 - Food Trucks
stl_merged.loc[stl_merged['Cluster Labels'] == 3, stl_merged.columns[[1] + list(range(5, stl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Denny-Blaine,Park,Monument / Landmark,Surf Spot,Beach,Fair,French Restaurant,Food Truck,Food & Drink Shop,Food,Flower Shop
23,View Ridge,Park,Bus Stop,Zoo,Fair,French Restaurant,Food Truck,Food & Drink Shop,Food,Flower Shop,Fish Market


In [90]:
# cluster5 - Beach and Zoo
stl_merged.loc[stl_merged['Cluster Labels'] == 4, stl_merged.columns[[1] + list(range(5, stl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Seward Park,Arts & Crafts Store,Beach,Zoo,Fair,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,Flower Shop
